In [2]:
# @name regulation
# @description notebook to build regulatory networks from various regulatory edges datasets with PMID from TRRUST and MSIGDB
# @author Núria Queralt Rosinach
# @date 01-17-2019

In [3]:
# to do:
#     * some edges have some resource without symbol. we need all symbols to retrieve concept
# attributes from biothings (eg, tftarget::tred, NCBIGene:9503 (retired entrez and replaced
# by another entrez, so it has symbol)). This is fixed here including 'retired' in the scopes cell 11
# for nodes file. It must be fixed in tftargets notebook. This only can be fixed for 'retired'
# Gene IDs, not for withdrawn Gene IDs.
#     * add namespace to entrez geneLists in tred, encode, neph, msigdb

In [4]:
import json
from biothings_client import get_client
import os, datetime
today = datetime.date.today()

if not os.path.exists('./tftargets/out'): os.makedirs('./tftargets/out')

In [5]:
def unique_list(dictionary, key, value):
    genes = dictionary.get(key,set())
    for gene in value:
        genes.add(gene)
    dictionary[key] = genes
    return dictionary

### Load TF-gene edges

In [6]:
tred = json.load(open('tftargets/data/tred.json'))
encode = json.load(open('tftargets/data/encode.json'))
neph2012 = json.load(open('tftargets/data/neph2012.json'))
trrust = json.load(open('tftargets/data/trrust.json'))
msigdb = json.load(open('msigdb/out/tf_genelist_entrez_msigdb.json'))

#### Neph2012

In [7]:
neph_all = dict()
for cell in neph2012:
    for tf, genes in neph2012[cell].items():
        neph_all = unique_list(neph_all, tf, genes)
neph = {key: list(neph_all[key]) for key in neph_all}

### Normalize 2 entrez, HGNC Id

* **symbols**: TF, trrust.genes 
* **entrez**: tred.genes, encode.genes, neph.genes, msigdb.genes

#### Input ID list: symbols

In [8]:
# symbols 
trrust_genes = { gene for tf in trrust for gene in trrust.get(tf) }
symbols = list(tred.keys() | encode.keys() | neph.keys() | trrust.keys() | msigdb.keys() | trrust_genes)
print('symbols:', len(symbols))

symbols: 3071


#### ID dictionaries: symbol2entrez, symbol2hgnc

In [9]:
# query biothings for symbol2entrez and symbol2hgnc 
mg = get_client('gene')
df = mg.querymany(symbols, scopes = 'symbol,alias', fields='entrezgene,HGNC', size=1, as_dataframe=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
53 input query terms found no hit:
	['CDPCR1', 'CEBPGAMMA', 'PTF1BETA', 'FOX', 'NFKAPPAB65', 'MEIS1AHOXA9', 'INSAF', 'TFIII', 'MYOGNF1',
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [10]:
df.head(2)

,HGNC,_id,_score,entrezgene,notfound
query,,,,,
PLOD1,9081,5351,86.69910,5351,NaN
CEBPG,1837,1054,85.44619,1054,NaN


In [11]:
# not found
missing = df[['notfound']].copy()
missing = missing.reset_index().rename(columns={'query': 'symbol'})
missing = missing[missing['notfound'] == True][['symbol']] 
missing_symbol_l = [ symbol for symbol in missing['symbol'] ]

# save not found
with open('./tftargets/out/not_found_symbols.list','w') as f:
    f.write('\n'.join(missing_symbol_l))

In [12]:
# prepare ids dataframe for dictionary construction
ids =( df.reset_index()
         .rename(columns={'query': 'symbol','HGNC': 'hgnc', 'entrezgene': 'entrez'}) 
         [['symbol','hgnc','entrez']]
         .copy()
     )

print('symbol:', type(ids['symbol'][0]))
print('hgnc:', type(ids['hgnc'][0]))
print('entrez:', type(ids['entrez'][0]))
ids.head(2)

symbol: <class 'str'>
hgnc: <class 'str'>
entrez: <class 'str'>


,symbol,hgnc,entrez
0,PLOD1,9081,5351
1,CEBPG,1837,1054


In [13]:
# example of not found value
ids[ids['symbol'] == 'ALPHACP1']

,symbol,hgnc,entrez
947,ALPHACP1,NaN,NaN


In [14]:
# example of found value
ids[ids['symbol'] == 'NPC1L1']

,symbol,hgnc,entrez
1796,NPC1L1,7898,29881


In [15]:
x = ids.at[491,'hgnc']
print('x: {}'.format(type(x)))
print('x: {}'.format(x))
if isinstance(x,float): 
    print('x: {}'.format(x))

x: <class 'str'>
x: 2917


In [16]:
# build dictionaries
# value types: 
# symbol=> str, no nulls
# hgnc=> str, null=> float(nan)
# entrez=> str, null=> float(0)
# dictionaries for ID mapping (nulls allowed). No namespace added.
#symbol2entrez_map = dict(zip(ids.symbol,ids.entrez))
#symbol2hgnc_map = dict(zip(ids.symbol,ids.hgnc))

# dictionaries for normalization to an ID (not null allowed). the final ID must be from one of the several schemes.
# namespace added.
symbol2entrez_dict = dict(zip(ids.symbol,ids.entrez))
symbol2hgnc_dict = dict(zip(ids.symbol,ids.hgnc))

# associate symbol for those genes without entrez: symbol => entrez > symbol
for symbol in symbol2entrez_dict:
    if isinstance(symbol2entrez_dict[symbol],float):
        symbol2entrez_dict[symbol] = symbol
    else: 
        entrez = symbol2entrez_dict[symbol]
        symbol2entrez_dict[symbol] = "NCBIGene:"+entrez

# associate entrez else symbol for those genes without hgnc: symbol =>  hgnc > entrez > symbol
for symbol in symbol2hgnc_dict:
    if isinstance(symbol2hgnc_dict.get(symbol),float):
        symbol2hgnc_dict[symbol] = symbol2entrez_dict[symbol]
    else:
        hgnc = symbol2hgnc_dict[symbol]
        symbol2hgnc_dict[symbol] = "HGNC:"+hgnc
        
# checks
# nor hgnc neither entrez associated
print(symbol2entrez_dict['ALPHACP1'])
print(symbol2hgnc_dict['ALPHACP1'])

# hgnc and entrez associated
print(symbol2entrez_dict['NPC1L1'])
print(symbol2hgnc_dict['NPC1L1'])

ALPHACP1
ALPHACP1
NCBIGene:29881
HGNC:7898


#### Normalize ID to entrez, else: symbol

In [17]:
# Map symbols to entrez
tred_entrez = { symbol2entrez_dict[symbol]: tred[symbol] for symbol in tred }
encode_entrez = { symbol2entrez_dict[symbol]: encode[symbol] for symbol in encode }
neph_entrez = { symbol2entrez_dict[symbol]: neph[symbol] for symbol in neph }
msigdb_entrez = { symbol2entrez_dict[symbol]: msigdb[symbol] for symbol in msigdb }
trrust_entrez = {}
for symbol in trrust: 
    genes_entrez = list()
    for gene in trrust.get(symbol):
        genes_entrez.append(symbol2entrez_dict[gene])
    trrust_entrez[symbol2entrez_dict[symbol]] =  genes_entrez
    
# save associations normalized to entrez
with open('./tftargets/out/tred_entrez.json', 'w') as f:
    json.dump(tred_entrez, f, sort_keys=True, indent=2)

with open('./tftargets/out/encode_entrez.json', 'w') as f:
    json.dump(encode_entrez, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/neph_entrez.json', 'w') as f:
    json.dump(neph_entrez, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/msigdb_entrez.json', 'w') as f:
    json.dump(msigdb_entrez, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/trrust_entrez.json', 'w') as f:
    json.dump(trrust_entrez, f, sort_keys=True, indent=2)

**Check**:
* neph and neph_entrez len are not equivalent, why? same with msigdb

In [18]:
# check map symbols to entrez
len(neph)

536

In [19]:
len(neph_entrez)

535

In [20]:
len(msigdb)

282

In [21]:
len(msigdb_entrez)

277

In [22]:
# what are the missing keys?
# check that 53 symbols without entrez,hgnc are in the dictionaries: 
print(neph.keys() & neph_entrez.keys())
print(msigdb.keys() & msigdb_entrez.keys())
print(len(msigdb.keys() & msigdb_entrez.keys()))
# yes, 52 + 'INSAF' 

# the missing values can be 2 symbols that share the same entrez because they are alias for the same gene

{'INSAF'}
{'HAND1E47', 'ALPHACP1', 'CACCCBINDINGFACTOR', 'TAXCREB', 'AP1FJ', 'COMP1', 'MEIS1BHOXA9', 'NKX22', 'E2F1DP1', 'TCF11MAFG', 'RORA1', 'TFIII', 'ISRE', 'PTF1BETA', 'CREBP1', 'CDPCR3', 'MYCMAX', 'AMEF2', 'GNCF', 'NFMUE1', 'CREL', 'TCF1P', 'MMEF2', 'NFKAPPAB65', 'CETS1P54', 'LMO2COM', 'CREBP1CJUN', 'MEIS1AHOXA9', 'FOX', 'CDPCR3HD', 'E2F4DP2', 'CEBPDELTA', 'T3R', 'CACBINDINGPROTEIN', 'TAL1ALPHAE47', 'HMEF2', 'COREBINDINGFACTOR', 'NKX25', 'IK3', 'MYOGNF1', 'CDPCR1', 'HP1SITEFACTOR', 'NFY', 'TAL1BETAE47', 'CEBPGAMMA', 'E2F4DP1', 'E2F1DP2', 'TAL1BETAITF2', 'CART1', 'E2F1DP1RB', 'NKX62', 'AHRARNT'}
52


In [23]:
# tred and encode look fine
len(tred) 

133

In [25]:
len(tred_entrez)

133

In [27]:
len(encode)

157

In [26]:
len(encode_entrez)

157

#### Input ID list: entrez

In [31]:
# entrez
tred_genes = { gene for tf in tred for gene in tred.get(tf) }
encode_genes = { gene for tf in encode for gene in encode.get(tf) }
neph_genes = { gene for tf in neph for gene in neph.get(tf) }
msigdb_genes = { gene for tf in msigdb for gene in msigdb.get(tf) }
entrez = list(tred_genes | encode_genes | neph_genes | msigdb_genes)
print('entrez:', len(entrez))

entrez: 16632


#### ID dictionaries: entrez2hgnc, entrez2symbol

In [32]:
# symbol2entrez dict
mg = get_client('gene')
df = mg.querymany(entrez, scopes = 'entrezgene', fields='HGNC,symbol', size=1, as_dataframe=True)
df.head(2)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-16632...done.
Finished.
137 input query terms found no hit:
	['400870', '338850', '221943', '101243544', '284072', '4276', '51036', '326332', '284211', '143902',
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


,HGNC,_id,_score,notfound,symbol
query,,,,,
6432,10789,6432,21.382607,NaN,SRSF7
57650,29302,57650,20.849570,NaN,CIP2A


In [33]:
# not found
missing = df[['notfound']].copy()
missing = missing.reset_index().rename(columns={'query': 'entrez'})
missing = missing[missing['notfound'] == True][['entrez']] 
missing_entrez_l = [ entrez for entrez in missing['entrez'] ]

# save not found
with open('./tftargets/out/not_found_entrez.list','w') as f:
    f.write('\n'.join(missing_entrez_l))

In [34]:
# prepare ids dataframe for dictionary construction
ids =( df.reset_index()
         .rename(columns={'query': 'entrez','HGNC': 'hgnc'}) 
         [['entrez','hgnc','symbol']]
         .copy()
     )
print('symbol:', type(ids['symbol'][0]))
print('hgnc:', type(ids['hgnc'][0]))
print('entrez:', type(ids['entrez'][0]))
ids['entrez_id'] = ids.entrez.apply(lambda x: "NCBIGene:"+x if type(x) == str else x)
ids['hgnc_id'] = ids.hgnc.apply(lambda x: "HGNC:"+x if type(x) == str else x)
ids.head(2)

symbol: <class 'str'>
hgnc: <class 'str'>
entrez: <class 'str'>


,entrez,hgnc,symbol,entrez_id,hgnc_id
0,6432,10789,SRSF7,NCBIGene:6432,HGNC:10789
1,57650,29302,CIP2A,NCBIGene:57650,HGNC:29302


In [35]:
# build dictionaries
# value types:
# entrez=> str, no nulls
# symbol=> str, null=> float(nan)
# hgnc=> str, null=> float(nan)
# dictionaries for normalization to an ID (not null allowed). the final ID must be from one of the several schemes.
# namespace added.
entrez2hgnc_dict = dict(zip(ids.entrez_id,ids.hgnc_id))
entrez2symbol_dict = dict(zip(ids.entrez_id,ids.symbol))

# associate entrez for those genes without hgnc: entrez=> hgnc > entrez
for entrez in entrez2hgnc_dict:
    if isinstance(entrez2hgnc_dict.get(entrez),float): 
        entrez2hgnc_dict[entrez] = entrez
        
# associate symbol, else 'NA' because it's a dict to annotate: entrez=> symbol or 'NA'
for entrez in entrez2symbol_dict:
    if isinstance(entrez2symbol_dict.get(entrez),float): 
        entrez2symbol_dict[entrez] = 'NA'

#### Normalize ID to hgnc, else: entrez/symbol


In [36]:
#### Map symbols,entrez to hgnc
tred_hgnc = { symbol2hgnc_dict[symbol]: tred[symbol] for symbol in tred }
encode_hgnc = { symbol2hgnc_dict[symbol]: encode[symbol] for symbol in encode }
neph_hgnc = { symbol2hgnc_dict[symbol]: neph[symbol] for symbol in neph }
msigdb_hgnc = { symbol2hgnc_dict[symbol]: msigdb[symbol] for symbol in msigdb }

trrust_hgnc = {}
for symbol in trrust: 
    genes_hgnc = list()
    for gene in trrust.get(symbol):
        genes_hgnc.append(symbol2hgnc_dict[gene])
    trrust_hgnc[symbol2hgnc_dict[symbol]] =  genes_hgnc

for tf in tred_hgnc: 
    genes_hgnc = list()
    for gene in tred_hgnc.get(tf):
        genes_hgnc.append(entrez2hgnc_dict["NCBIGene:"+str(gene)])
    tred_hgnc[tf] =  genes_hgnc
for tf in encode_hgnc: 
    genes_hgnc = list()
    for gene in encode_hgnc.get(tf):
        genes_hgnc.append(entrez2hgnc_dict["NCBIGene:"+str(gene)])
    encode_hgnc[tf] =  genes_hgnc
for tf in neph_hgnc: 
    genes_hgnc = list()
    for gene in neph_hgnc.get(tf):
        genes_hgnc.append(entrez2hgnc_dict["NCBIGene:"+str(gene)])
    neph_hgnc[tf] =  genes_hgnc
for tf in msigdb_hgnc: 
    genes_hgnc = list()
    for gene in msigdb_hgnc.get(tf):
        genes_hgnc.append(entrez2hgnc_dict["NCBIGene:"+str(gene)])
    msigdb_hgnc[tf] =  genes_hgnc
    
# save associations normalized to hgnc
with open('./tftargets/out/tred_hgnc.json', 'w') as f:
    json.dump(tred_hgnc, f, sort_keys=True, indent=2)

with open('./tftargets/out/encode_hgnc.json', 'w') as f:
    json.dump(encode_hgnc, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/neph_hgnc.json', 'w') as f:
    json.dump(neph_hgnc, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/msigdb_hgnc.json', 'w') as f:
    json.dump(msigdb_hgnc, f, sort_keys=True, indent=2)
    
with open('./tftargets/out/trrust_hgnc.json', 'w') as f:
    json.dump(trrust_hgnc, f, sort_keys=True, indent=2)

### Save edges

Edge format:

| source | dataset | tf_source_id | gene_source_id | source_uri | s_entrez_id | s_hgnc_id | s_symbol | p_id | p_label | o_entrez_id | o_hgnc_id | o_symbol | reference_id | reference_date | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|

1. `source`: str tftargets
2. `dataset`: str tred 
3. `tf_source_id`: str transcriptor factor tftargets id
4. `gene_source_id`: str gene tftargets id 
5. `source_uri`: str 'https://github.com/slowkow/tftargets' 
6. `s_entrez_id`: str subject entrez 
7. `s_hgnc_id`: str subject hgnc id 
8. `s_symbol`: str subject label 
9. `p_id`:  str property id 'RO:0002434'
10. `p_label`: str property label 'interacts with'
11. `o_entrez_id`: str object entrez 
12. `o_hgnc_id`: str object hgnc id 
13. `o_symbol`: str object label  
14. `reference_id`: str pmid 
15. `reference_date`: str yyyy-mm-dd 

In [37]:
# add ref_uri to trrust and msigdb statements
# trrust: dict {'tf:gene': 'PMID:'}
import gzip

def add_elem_dictionary(dictionary, key, elem, repet = False):
    if key in dictionary:
        aux = dictionary.get(key)
        if repet:
            aux.append(elem)
            dictionary[key] = aux
        else:   
            if not elem in aux:
                aux.append(elem)
                dictionary[key] = aux
    else:
        dictionary[key] = [elem]
    return dictionary

st2ref = {}
for line in gzip.open('./tftargets/data-raw/TRRUST/trrust_rawdata.txt.gz', 'rt').readlines():
    tf,gene,mor,ref_list = line.strip().split('\t')
    st = tf+':'+gene
    ref_list = ref_list.split(';') if ';' in ref_list else [ref_list]
    for ref in ref_list:
        add_elem_dictionary(st2ref, st, ref)

for st in st2ref:
    #st2ref[st] = 'https://www.ncbi.nlm.nih.gov/pubmed/'+','.join(st2ref[st])
    st2ref[st] = 'PMID:'+';'.join(st2ref[st])

In [38]:
# tftargets: tred, encode, neph, trrust
with open('./tftargets/out/tftargets_edges.csv','w') as f:
    f.write("source,dataset,tf_source_id,gene_source_id,source_uri,s_entrez_id,s_hgnc_id,s_symbol,p_id,p_label,o_entrez_id,o_hgnc_id,o_symbol,reference_id,reference_date\n")
    source = "tftargets"
    source_uri = "https://github.com/slowkow/tftargets"
    p_id = "RO:0002434"
    p_label = "interacts with"
    # tred
    dataset = "tred"
    reference_id = "PMID:17202159"
    reference_date = "2007-01-01"
    for tf in tred:
        for gene in tred[tf]:
            f.write(
                "{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n"
                .format(
                    source,
                    dataset,
                    tf,
                    gene,
                    source_uri,
                    symbol2entrez_dict[tf],
                    symbol2hgnc_dict[tf],
                    tf,
                    p_id,
                    p_label,
                    "NCBIGene:"+str(gene),
                    entrez2hgnc_dict["NCBIGene:"+str(gene)],
                    entrez2symbol_dict["NCBIGene:"+str(gene)],
                    reference_id,
                    reference_date
            )
        )
    # encode
    dataset = "encode_ENCFF001UUQ"
    reference_id = "ENCODE:ENCFF001UUQ"
    reference_date = "2012-08-28"
    for tf in encode:
        for gene in encode[tf]:
            f.write(
                "{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n"
                .format(
                    source,
                    dataset,
                    tf,
                    gene,
                    source_uri,
                    symbol2entrez_dict[tf],
                    symbol2hgnc_dict[tf],
                    tf,
                    p_id,
                    p_label,
                    "NCBIGene:"+str(gene),
                    entrez2hgnc_dict["NCBIGene:"+str(gene)],
                    entrez2symbol_dict["NCBIGene:"+str(gene)],
                    reference_id,
                    reference_date
            )
        )
    # neph
    dataset = "neph2012"
    reference_id = "PMID:22959076"
    reference_date = "2012-09-14"
    for tf in neph:
        for gene in neph[tf]:
            f.write(
                "{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n"
                .format(
                    source,
                    dataset,
                    tf,
                    gene,
                    source_uri,
                    symbol2entrez_dict[tf],
                    symbol2hgnc_dict[tf],
                    tf,
                    p_id,
                    p_label,
                    "NCBIGene:"+str(gene),
                    entrez2hgnc_dict["NCBIGene:"+str(gene)],
                    entrez2symbol_dict["NCBIGene:"+str(gene)],
                    reference_id,
                    reference_date
            )
        )
    # trrust
    dataset = "trrust"
    #reference_id = "PMID:26066708"
    reference_date = "NA" #"2015-06-12"
    for tf in trrust:
        for gene in trrust[tf]:
            reference_id = st2ref[tf+':'+gene]
            f.write(
                "{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n"
                .format(
                    source,
                    dataset,
                    tf,
                    gene,
                    source_uri,
                    symbol2entrez_dict[tf],
                    symbol2hgnc_dict[tf],
                    tf,
                    p_id,
                    p_label,
                    symbol2entrez_dict[gene],
                    symbol2hgnc_dict[gene],
                    gene,
                    reference_id,
                    reference_date
            )
        )        

In [39]:
# msigdb: dict {'tf:gene': 'reference'}
unknown = list()
unrecognized = list()
st2ref = {}
for line in open("./msigdb/data/c3.tft.v6.1.entrez.gmt").readlines():
    geneset_name = line.strip().split('\t')[0]
    gs_name_v = geneset_name.split('_')
    # gene symbols: Ideally, symbols should be no longer than six characters in length. 
    # GCTNWTTGK_UNKNOWN
    if 'unknown' in gs_name_v[-1].lower():
        unknown.append(geneset_name)
        continue
    # LEN = 4 GATTGGY_NFY_Q6_01
    elif len(gs_name_v) == 4:
        tf = gs_name_v[1]
    # LEN = 3 two tfids: motif+TFACID or TFACID
    # GCCATNTTG_YY1_Q6
    elif len(gs_name_v) == 3 and len(gs_name_v[0]) >= 6:
        tf = gs_name_v[1]
    # AP4_Q6_01
    elif len(gs_name_v) == 3 and len(gs_name_v[0]) < 6:
        tf = gs_name_v[0]
    # LEN = 2 GFI1_01
    elif len(gs_name_v) == 2:
        tf = gs_name_v[0]
    else:
        unrecognized.append(geneset_name)
        continue
    ref_uri = line.strip().split('\t')[1]
    genelist = line.strip().split('\t')[2:]
    for gene in set(genelist):
        st = tf+':'+gene
        add_elem_dictionary(st2ref, st, ref_uri)
    
# convert reference_uri list to str    
for st in st2ref:
    if len(st2ref[st]) > 1:
        st2ref[st] = '|'.join(st2ref[st])
        #print(st, st2ref[st])
    else:
        #st2ref[st] = st2ref[st]
        for ref in st2ref[st]:
            st2ref[st] = ref

In [40]:
# msigdb
if not os.path.exists('./msigdb/out'): os.makedirs('./msigdb/out')
with open('./msigdb/out/msigdb_edges.csv','w') as f:
    f.write("source,dataset,tf_source_id,gene_source_id,source_uri,s_entrez_id,s_hgnc_id,s_symbol,p_id,p_label,o_entrez_id,o_hgnc_id,o_symbol,reference_id,reference_date\n")
    source = "msigdb"
    source_uri = "http://software.broadinstitute.org/gsea/msigdb"
    p_id = "RO:0002434"
    p_label = "interacts with"
    # c3:tft
    dataset = "c3:tft"
    #reference_id = "NA"
    reference_date = "NA"
    for tf in msigdb:
        for gene in msigdb[tf]:
            reference_id = st2ref[tf+':'+gene]
            f.write(
                "{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n"
                .format(
                    source,
                    dataset,
                    tf,
                    gene,
                    source_uri,
                    symbol2entrez_dict[tf],
                    symbol2hgnc_dict[tf],
                    tf,
                    p_id,
                    p_label,
                    "NCBIGene:"+str(gene),
                    entrez2hgnc_dict["NCBIGene:"+str(gene)],
                    entrez2symbol_dict["NCBIGene:"+str(gene)],
                    reference_id,
                    reference_date
            )
        )

### Prepare Regulation edges to build the graph
* concat tftargets and msigdb
* give graph format
* build edges and nodes files
* concat with graph (edges and nodes): first curated > monarch > regulation
* drop duplicates ((edges/nodes) rows, concepts
* save graph
* format for neo4j
* save neo4j files

In [41]:
import pandas as pd

#### concat tftargets and msigdb

In [42]:
# tftargets
tftargets = pd.read_csv('./tftargets/out/tftargets_edges.csv')
print(tftargets.shape)
tftargets.tail(2)

(83310, 15)


/home/nuria/anaconda3/envs/avalanche/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source,dataset,tf_source_id,gene_source_id,source_uri,s_entrez_id,s_hgnc_id,s_symbol,p_id,p_label,o_entrez_id,o_hgnc_id,o_symbol,reference_id,reference_date
83308,tftargets,trrust,ZNRD1,VEGFA,https://github.com/slowkow/tftargets,NCBIGene:30834,HGNC:13182,ZNRD1,RO:0002434,interacts with,NCBIGene:7422,HGNC:12680,VEGFA,PMID:21080911,NaN
83309,tftargets,trrust,ZSCAN21,SNCA,https://github.com/slowkow/tftargets,NCBIGene:7589,HGNC:13104,ZSCAN21,RO:0002434,interacts with,NCBIGene:6622,HGNC:11138,SNCA,PMID:19549071,NaN


In [43]:
# msigdb
msigdb = pd.read_csv('./msigdb/out/msigdb_edges.csv')
print(msigdb.shape)
msigdb.head(2)

(114157, 15)


,source,dataset,tf_source_id,gene_source_id,source_uri,s_entrez_id,s_hgnc_id,s_symbol,p_id,p_label,o_entrez_id,o_hgnc_id,o_symbol,reference_id,reference_date
0,msigdb,c3:tft,AFP1,10911,http://software.broadinstitute.org/gsea/msigdb,NCBIGene:843257,NCBIGene:843257,AFP1,RO:0002434,interacts with,NCBIGene:10911,HGNC:12636,UTS2,http://www.broadinstitute.org/gsea/msigdb/card...,NaN
1,msigdb,c3:tft,AFP1,9241,http://software.broadinstitute.org/gsea/msigdb,NCBIGene:843257,NCBIGene:843257,AFP1,RO:0002434,interacts with,NCBIGene:9241,HGNC:7866,NOG,http://www.broadinstitute.org/gsea/msigdb/card...,NaN


In [44]:
# concat tftargets and msigdb tg-gene edges
edges = pd.concat([tftargets,msigdb],ignore_index=True)
print(edges.shape)

(197467, 15)


In [45]:
# see duplicates
len(edges[edges.duplicated(keep=False)])

400

In [46]:
# drop duplicates
edges.drop_duplicates(inplace=True)
print(len(edges))

197267


#### build edges and nodes file
file format: csv
fill null wiht 'NA'. can be done with python function before saving
##### edges
1. `subject_id` str curie required
2. `object_id` str curie required
3. `property_id` str curie NA
4. `property_label` str NA
5. `property_description` str NA
6. `property_uri` str NA
7. `reference_uri` str NA
8. `reference_supporting_text` str NA
9. `reference_date` str NA | yyyy-mm-dd

In [47]:
import os
if not os.path.exists('./graph'): os.makedirs('./graph')

#### select gene ID

In [48]:
# select id
curie = 'hgnc'.lower()

# give graph format 
print(edges.columns)
edges = (   edges
            .rename(columns={'s_hgnc_id': 'subject_id',
                             'p_id': 'property_id',
                             'p_label': 'property_label',
                             'o_hgnc_id': 'object_id'
                            })
)

curie_dct = {
    'ro': 'http://purl.obolibrary.org/obo/',
    'pmid': 'https://www.ncbi.nlm.nih.gov/pubmed/',
    'encode': 'https://www.encodeproject.org/search/?searchTerm='
}

edges_l = list()
for i, row in edges.iterrows():
    # property uri: http://purl.obolibrary.org/obo/RO_0002434
    property_uri = 'NA'
    if ':' in row['property_id']:
        property_uri = curie_dct[row['property_id'].split(':')[0].lower()]+row['property_id'].replace(':','_')       
    
    # reference_uri: https://www.ncbi.nlm.nih.gov/pubmed/25416956
    # capture nan or None values, i.e. all possible nulls
    #if (isinstance(row['reference_id'], float) and str(row['reference_id']).lower() == 'nan') or row['reference_id'] is None:
    #    row['reference_id'] = 'NA'
    if ':' not in str(row['reference_id']):
        reference_uri = row['source_uri'] #row['reference_id']
    elif 'http://www.broadinstitute.org/gsea/msigdb/cards/' in str(row['reference_id']):
        reference_uri = row['reference_id']
    else:
        try:
            reference_uri = curie_dct[row['reference_id'].split(':')[0].lower()]+row['reference_id'].split(':')[1].replace(';',',') 
            #if 'pmid' in row['reference_id'].lower(): print(reference_uri)
        except KeyError:
            reference_uri = row['reference_id']
            print('There is a reference curie with and unrecognized namespace:', row['reference_id'])
    # build list of edges as list of dict, i.e a df, where a dict is an edge        
    edge = dict()
    edge['subject_id'] = row['subject_id']
    edge['object_id'] = row['object_id']
    edge['property_id'] = row['property_id']
    edge['property_label'] = row['property_label']
    edge['property_description'] = 'NA'
    edge['property_uri'] = property_uri
    edge['reference_uri'] = reference_uri
    edge['reference_supporting_text'] = 'This edge comes from the {} dataset in "{}" source.'.format(row['dataset'].upper(),row['source']) # 'NA'
    edge['reference_date'] = row['reference_date']
    edges_l.append(edge)

# save edges file
pd.DataFrame(edges_l).fillna('NA').to_csv('./graph/regulation_edges_v{}.csv'.format(today), index=False)

Index(['source', 'dataset', 'tf_source_id', 'gene_source_id', 'source_uri',
       's_entrez_id', 's_hgnc_id', 's_symbol', 'p_id', 'p_label',
       'o_entrez_id', 'o_hgnc_id', 'o_symbol', 'reference_id',
       'reference_date'],
      dtype='object')


##### nodes
1. `id` str curie required
2. `semantic_groups` str required
3. `preflabel` str label required
4. `synonyms` str NA
5. `description` str NA
6. **new** `name` str NA

In [49]:
print(len(edges))

197267


In [50]:
# retrieve node attributes from biothings and build dictionary
# from biothings we retrieve: name (new attribute for short description), alias (synonyms), summary (description)
# symbols in this case come from the original source. otherwise are gonna be retrieved from biothings as well.
# build concept dict: {id:symbol}
concept_dct = dict()
for i, row in edges.iterrows():
    # node for subject
    concept_dct[row['subject_id']] = {
        'preflabel': row['s_symbol'],
        'name': None,
        'synonyms': None,
        'description': None
    }
    # node for object
    concept_dct[row['object_id']] = {
        'preflabel': row['o_symbol'],
        'name': None,
        'synonyms': None,
        'description': None
    }
len(concept_dct.keys())

16963

### Trap genes without symbol

In [51]:
# trap nan symbols, i.e. discontinued entrez gene id
concepts_no_symbol_set = set()
for concept in concept_dct:
    if str(concept_dct[concept]['preflabel']) == 'nan':
        concepts_no_symbol_set.add(concept)
print(len(concepts_no_symbol_set))
concepts_no_symbol_set
concepts_l = list(concepts_no_symbol_set)
pd.DataFrame({'id':concepts_l}).id.apply(lambda x: x.split(':')[0]).value_counts()
# checked that all are entrez

137


NCBIGene    137
Name: id, dtype: int64

In [52]:
# retrieve symbol for every ncbi gene
entrez = list()
for concept in concept_dct:
    if str(concept_dct[concept]['preflabel']) == 'nan':
        entrez.append(concept.split(':')[1])

mg = get_client('gene')
df1 = mg.querymany(entrez, scopes = 'entrezgene,retired', fields='symbol', size=1, as_dataframe=True)
df1.head(2)

querying 1-137...done.
Finished.
88 input query terms found no hit:
	['79907', '79911', '93333', '121301', '143902', '146856', '151720', '197379', '219392', '221943', '2
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


,_id,_score,notfound,symbol
query,,,,
9503,653067,1.55,NaN,XAGE1B
9714,55619,1.55,NaN,DOCK10


In [53]:
print(df1.shape)
print(len(set(df1.symbol)))
print(df1.symbol.isna().sum())
print('88 entrez have no hit, i.e. withdrawn entrez from NCBI Gene')

(137, 4)
49
88
88 entrez have no hit, i.e. withdrawn entrez from NCBI Gene


In [54]:
# some entrez don't exist anymore >> no attributes
# build ncbi2symbol dictionary
e2s_df = df1.reset_index().rename(columns={'query':'entrez'}).copy()
e2s = dict(zip(e2s_df.entrez, e2s_df.symbol))
print(len(e2s_df.entrez),len(set(e2s_df.entrez)))
print(len(e2s_df.symbol),len(set(e2s_df.symbol)))
print(len(e2s))
# dict stores nan values

137 137
137 49
137


In [55]:
for concept in concept_dct:
    if str(concept_dct[concept]['preflabel']) == 'nan' and concept.split(':')[0] == 'NCBIGene':
        concept_dct[concept]['preflabel'] = e2s[concept.split(':')[1]]

In [56]:
# biothings api + dictionaries
# input list for api: since by id we have hgnc/entrez or symbol, i am gonna use symbol
symbols = list()
symbols2 = list()
for idx,symbol in concept_dct.items():
    #id = key.split(':')[1] if ':' in key else key
    if str(symbol['preflabel']) != 'nan':
        symbols.append(symbol['preflabel']) 
    # withdrawn entrez
    else:
        symbols2.append(symbol['preflabel']) 
    
print(symbols[0:5])    
print('# of unique concepts with symbol:',len(symbols))
print('# of unique symbols:',len(set(symbols)))
print()
print(symbols2[0:5])    
print('# of unique concepts without symbol, i.e. withdrawn:',len(symbols2))
print('# of unique symbols:',len(set(symbols2)))

['PAX1', 'PDGFRA', 'VHL', 'SLC22A6', 'SLC22A8']
# of unique concepts with symbol: 16875
# of unique symbols: 16843

[nan, nan, nan, nan, nan]
# of unique concepts without symbol, i.e. withdrawn: 88
# of unique symbols: 1


In [57]:
# api call
symbols = list(set(symbols))
mg = get_client('gene')
df = mg.querymany(symbols, scopes = 'symbol,alias', fields='alias,name,summary', size=1, as_dataframe=True)
df.head(2)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-16843...done.
Finished.
53 input query terms found no hit:
	['CEBPGAMMA', 'NFKAPPAB65', 'MEIS1AHOXA9', 'INSAF', 'ISRE', 'E2F1DP2', 'CDPCR3HD', 'E2F4DP2', 'RORA1
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


,_id,_score,alias,name,notfound,summary
query,,,,,,
ELOA-AS1,100506963,107.30683,TCEB3-AS1,ELOA antisense RNA 1,NaN,NaN
MYO18B,84700,85.55801,KFS4,myosin XVIIIB,NaN,The protein encoded by this gene may regulate ...


In [58]:
print(df.shape)
print(len(concept_dct.keys()))

(16843, 6)
16963


In [59]:
# dictionaries {id: {name:, alias:, summary:}}
i = 0
print(len(concept_dct))
for symbol, row in df.iterrows():
    # associate concept to symbol
    for concept in concept_dct:
        #print(concept, symbol, concept_dct[concept]['preflabel'], row)
        # 88 concepts without symbol (16992-88=16904 with symbol)
        if concept_dct[concept]['preflabel'] == symbol:
            i += 1
            # add attributes
            #print(concept, symbol, row['name'], row)
            concept_dct[concept]['name'] = row['name']
            concept_dct[concept]['synonyms'] = row['alias']
            concept_dct[concept]['description'] = row['summary']
print(i)
print(len(concept_dct))

16963
16875
16963


In [60]:
# build a list of nodes as list of dict, i.e a df, where a dict is a node
nodes_l = list()
for concept in concept_dct:
    # node for subject
    node = dict()
    node['id'] = concept
    node['semantic_groups'] = 'GENE'
    node['preflabel'] = concept_dct[concept]['preflabel']
    node['name'] = concept_dct[concept]['name']
    node['synonyms'] = '|'.join(list(concept_dct[concept]['synonyms'])) if isinstance(concept_dct[concept]['synonyms'], list) else concept_dct[concept]['synonyms']
    node['description'] = concept_dct[concept]['description']
    nodes_l.append(node)
    
# save nodes file    
pd.DataFrame(nodes_l).fillna('NA').to_csv('./graph/regulation_nodes_v{}.csv'.format(today), index=False)
len(nodes_l)

16963

In [61]:
len(edges_l)

197267

In [62]:
import pandas as pd
df = pd.read_csv('./graph/regulation_edges_v{}.csv'.format(today))
df.tail(5)

/home/nuria/anaconda3/envs/avalanche/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,object_id,property_description,property_id,property_label,property_uri,reference_date,reference_supporting_text,reference_uri,subject_id
197262,HGNC:10931,NaN,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434,NaN,"This edge comes from the C3:TFT dataset in ""ms...",http://www.broadinstitute.org/gsea/msigdb/card...,HGNC:16764
197263,HGNC:24561,NaN,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434,NaN,"This edge comes from the C3:TFT dataset in ""ms...",http://www.broadinstitute.org/gsea/msigdb/card...,HGNC:16764
197264,HGNC:11209,NaN,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434,NaN,"This edge comes from the C3:TFT dataset in ""ms...",http://www.broadinstitute.org/gsea/msigdb/card...,HGNC:16764
197265,HGNC:26314,NaN,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434,NaN,"This edge comes from the C3:TFT dataset in ""ms...",http://www.broadinstitute.org/gsea/msigdb/card...,HGNC:16764
197266,HGNC:26137,NaN,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434,NaN,"This edge comes from the C3:TFT dataset in ""ms...",http://www.broadinstitute.org/gsea/msigdb/card...,HGNC:16764


In [63]:
import pandas as pd
df = pd.read_csv('./graph/regulation_nodes_v{}.csv'.format(today))
df.head(5)

,description,id,name,preflabel,semantic_groups,synonyms
0,This gene is a member of the paired box (PAX) ...,HGNC:8615,paired box 1,PAX1,GENE,HUP48|OFC2
1,This gene encodes a cell surface tyrosine kina...,HGNC:8803,platelet derived growth factor receptor alpha,PDGFRA,GENE,CD140A|PDGFR-2|PDGFR2
2,Von Hippel-Lindau syndrome (VHL) is a dominant...,HGNC:12687,von Hippel-Lindau tumor suppressor,VHL,GENE,HRCA1|RCA1|VHL1|pVHL
3,The protein encoded by this gene is involved i...,HGNC:10970,solute carrier family 22 member 6,SLC22A6,GENE,HOAT1|OAT1|PAHT|ROAT1
4,This gene encodes a protein involved in the so...,HGNC:10972,solute carrier family 22 member 8,SLC22A8,GENE,OAT3


In [67]:
print(df.shape)
print(len(list(df.id)))
print(len(set(list(df.id))))

(16963, 6)
16963
16963
